# Quantum Fourier Transform
### In this notebook we define the Quantum Fourier Transform (QFT) and implement it on a quantum computer. This operation is of great importance since it is at the heart of most quantum algorithms which provide significant speed-up over their classical counterparts.

## 1. Definition
### The QFT can be viewed as the quantum version of the discrete Fourier transform (DFT), which is already widely used in digital signal and image processing. The $N$-dimensional DFT is a unitary transformation which acts on a vector $\mathbf{x} = (x_0, \dots, x_{N-1})$ and maps it to the vector $\mathbf{y} = (y_0, \dots, y_{N-1})$, with the components related by: $$y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_j\exp\left(\frac{2\pi i}{N}jk\right).$$
### Extending this definition analogously to the QFT (by simply using the ket vector notation), we arrive at the following unitary transformation $QFT_N$ acting on $n = \log_2(N)$ qubit states (we assume $N$ is an exact power of $2$): $$QFT_N\ket{x} = \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1}\exp\left(\frac{2\pi i}{N}xy\right)\ket{y},$$
### where $\ket{x}$ is a computational basis state in the $n$-qubit vector space.

## 2. Computation of QFT
### The real benefit of using the QFT in quantum algorithms is that, despite representing a unitary transformation on a vector space with dimensionality which depends exponentially on the number of qubits, it can be performed using a polynomial number of operations. This is due to the fact that the output state of the transformation when acting on a computational basis state is a product state, which we will now prove. We can rewrite the equation above as so: $\begin{align}QFT_N\ket{x} &= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1}\exp\left(\frac{2\pi i}{N}xy\right)\ket{y} \\ &= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1}\exp\left(\frac{2\pi i}{2^n}xy\right)\ket{y} \\ &= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1}\exp\left(2\pi ix\sum_{k=1}^{n}\frac{y_k}{2^k}\right)\ket{y = (y_1\dots y_n)_2} \\ &= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1}\prod_{k=1}^n\exp\left(2\pi i x\frac{y_k}{2^k}\right)\ket{y = (y_1\dots y_n)_2} \\ &= \frac{1}{\sqrt{N}}\prod_{k=1}^n\left(\ket{0}_k + \exp\left(2\pi ix\frac{1}{2^k}\right)\ket{1}_k\right) \\ &= \frac{1}{\sqrt{N}}\left(\ket{0} + \exp\left(2\pi ix\frac{1}{2}\right)\ket{1}\right)\otimes\dots\otimes\left(\ket{0} + \exp\left(2\pi ix\frac{1}{2^n}\right)\ket{1}\right)\end{align}$

### The way to implement this product-to-product state transformation is through the following steps. Let us have the state $\ket{x} = \ket{x_1\dots x_n}$. We apply a Hadamard transform to the left-most qubit, which we will specify as the first from now on, and get the state $$\hat{H}_1\ket{x_1\dots x_n} = \frac{1}{\sqrt{2}}\big(\ket{0} + (-1)^{x_1} \ket{1}\big)\ket{x_2\dots x_n} = \frac{1}{\sqrt{2}}\left(\ket{0} + \exp\left(2\pi i\frac{x_1}{2}\right)\ket{1}\right)\ket{x_2\dots x_n}.$$

### We then apply a controled phase rotation (CP) gate with the second qubit as the control and the first as the target. The angle of the phase rotation is $\theta_n = \frac{2\pi}{2^n}$ with $n=2.$ This transforms the state into $$CP^1_2(\theta_2)\hat{H}_1\ket{x_1\dots x_n} = \frac{1}{\sqrt{2}}\left(\ket{0} + \exp\left(2\pi i\frac{x_1}{2} + 2\pi i\frac{x_2}{2^2}\right)\ket{1}\right)\ket{x_2\dots x_n}.$$

### Repeating this process using each of the remaining qubits as the control qubit for a CP gate with the first qubit as the target, we get the state $\begin{align}CP^1_n(\theta_n)\dots CP^1_2(\theta_2)\hat{H}_1\ket{x_1\dots x_n} &= \frac{1}{\sqrt{2}}\left(\ket{0} + \exp\left(2\pi i\frac{x_1}{2} + \dots + 2\pi i\frac{x_n}{2^n}\right)\ket{1}\right)\ket{x_2\dots x_n}\\ &= \frac{1}{\sqrt{2}}\left(\ket{0} + \exp\left(2\pi ix\frac{1}{2^n}\right)\ket{1}\right)\ket{x_2\dots x_n},\end{align}$ where we have used that $x = x_12^{n-1} + \dots + x_n2^0.$ We see that the first qubit state is exactly the same as the last qubit state in the mathematical manipulation of the QFT above. To get the remaining qubits into the corresponding states, we repeat the operation described in an analogous way. To illustrate for the second qubit in $\ket{x}$, we apply a Hadamard transform to it, followed by $n-2$ CP gates with each of the remaining qubits as control gates in turn. This will generate the second-to-last qubit in the manipulation from before.
### Overall, we get the correct state using this approach, but with the qubits in reverse order. Hence, we just need to apply $\left\lfloor{\frac{n}{2}}\right\rfloor$ swap operations. The total number of gates we need to apply is $n + (n-1) + \dots + 1 + 3\left\lfloor{\frac{n}{2}}\right\rfloor = O(n^2)$.